# ⚙️ Analyse des Opportunités d'Automatisation

## Objectif
Identifier et prioriser les processus opérationnels à automatiser dans les cliniques.

---

## 📚 1. Imports et Configuration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

pd.set_option('display.max_columns', None)
print("✅ Imports réussis")

✅ Imports réussis


## 📂 2. Chargement des Données

In [2]:
df = pd.read_csv('../../donnees_cliniques_nettoyees.csv')
print(f"📊 {df.shape[0]} cliniques, {df.shape[1]} colonnes")
df.head(3)

📊 28 cliniques, 129 colonnes


,survey_start_datetime,survey_end_datetime,clinic_name,location,number_of_beds,number_of_healthcare_staff,medical_specialties,has_informatic_management_system,software_type_used,administrative_tasks_free_text,time_consuming_appointments,time_consuming_patient_records,time_consuming_billing_payment,time_consuming_inventory_orders,time_consuming_other_flag,time_consuming_other_details,main_challenges_patient_records_free_text,challenge_data_centralization,challenge_accessing_info,challenge_data_entry_errors,challenge_other_patient_records_flag,autres.1,uses_digital_tools_for_appointments,processes_to_automate_free_text,want_automation_appointments,want_automation_patient_records,want_automation_billing,want_automation_data_analysis,want_automation_inventory,want_automation_other_flag,autres.2,current_digital_channels_free_text,uses_website,uses_social_media,uses_newsletters,uses_other_channel_flag,autres.3,Site web,autre,RESEAUX SOCIAUX,autre.1,NEWSLETTERS,autre.2,SITE WEB,content_website_health_tips,content_website_prof_articles,content_website_deep_dive,content_website_patient_info,content_website_services_reviews,content_website_other_flag,autre.3,RESEAUX SOCIAUX.1,content_social_health_tips,content_social_prof_articles,content_social_deep_dive,content_social_patient_info,content_social_services_reviews,content_social_other_flag,autre.4,NEWSLETTERS.1,content_newsletter_health_tips,content_newsletter_prof_articles,content_newsletter_deep_dive,content_newsletter_patient_info,content_newsletter_services_reviews,content_newsletter_other_flag,autre.5,has_formal_digital_strategy,digital_strategy_outline,wants_help_with_strategy,has_dedicated_digital_budget,digital_communication_goals_free_text,goal_brand_awareness,goal_patient_acquisition,goal_patient_retention,goal_other_comm_flag,autre.6,digital_metrics_used,specific_online_expectations,has_dedicated_digital_team,missing_internal_digital_skills,difficulty_producing_content,main_digital_comm_difficulties,faced_compliance_issues,limited_by_time_or_staff,perceived_impact_digital_comm,level_staff_involved_digital_comm,patient_feedback_received,how_monitor_digital_practices,strategy_digital_innovation,adaptation_to_patient_online_expec,considered_IA_solutions,expected_IA_benefits_free_text,expect_IA_admin_efficiency,expect_IA_accuracy_improvement,expect_IA_resource_optimization,expect_IA_better_patient_experience,expect_IA_other_benefit_flag,autre.7,main_IA_adoption_barriers_free_text,barrier_IA_cost,barrier_IA_lack_of_skills,barrier_IA_security_privacy,barrier_IA_resistance_change,barrier_IA_other_flag,autre.8,interested_in_IA_demo_pilot,desired_IA_support_free_text,support_IA_training,support_IA_tech_assistance,support_IA_digital_consulting,support_IA_other_flag,autre.9,specific_IA_suggestions_expectations,thank_you_note,record_id,record_uuid,submission_time,validation_status,notes,status,submitted_by,form_version,tags,row_index,survey_duration_seconds,submission_year,submission_month,submission_day_of_week
0,2025-09-28 23:29:23.529,2025-09-28 23:42:24.873,PISAM,Cocody,123.0,NaN,NaN,Oui,SIH,prise de rendez-vous gestion des dossiers pati...,1.0,1.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Oui, un logiciel dédié",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,site web réseaux sociaux,1.0,1.0,0.0,0.0,NaN,autre,Site en refonte,2 à 3 fois,NaN,autre,En pause,"conseil santé prévention, actualités cliniques...",1.0,1.0,0.0,1.0,1.0,0.0,NaN,"conseils santé prévention, actualités clinique...",1.0,1.0,0.0,1.0,1.0,0.0,NaN,"Conseils santé prévention, actualités clinique...",1.0,1.0,0.0,1.0,1.0,0.0,NaN,oui,NaN,NaN,non,amélioration de la notoriété acquisition de la...,1.0,1.0,0.0,0.0,NaN,"Nombre de vues, engagement, consommation. Lois...",NaN,Oui,Intelligence artificielle,Non,Ressources humaines,Publicité médical,Les deux,Impact positif,09/10 ils participent à toutes les actions,Positif,NaN,NaN,NaN,oui,gain de temps et réduction des tâches administ...,1.0,1.0,1.0,0.0,0.0,NaN,autre,0.0,0.0,0.0,0.0,1.0,NaN,"peut-êt

## ⏱️ 3. Analyse des Processus Chronophages

In [3]:
# Colonnes des tâches chronophages
tache_columns = {
    'time_consuming_appointments': 'Prise de rendez-vous',
    'time_consuming_patient_records': 'Gestion dossiers patients',
    'time_consuming_billing_payment': 'Facturation et paiements',
    'time_consuming_inventory_orders': 'Gestion stocks/commandes'
}

def convert_to_numeric(val):
    if pd.isna(val):
        return 0
    val_str = str(val).lower().strip()
    return 1 if val_str in ['1', '1.0', 'oui', 'yes'] else 0

tache_data = []
for col, label in tache_columns.items():
    if col in df.columns:
        count = df[col].apply(convert_to_numeric).sum()
        pct = (count / len(df)) * 100
        tache_data.append({
            'Processus': label,
            'Nb Cliniques': int(count),
            'Pourcentage': pct
        })

df_taches = pd.DataFrame(tache_data).sort_values('Pourcentage', ascending=False)
print("\n⏱️ PROCESSUS LES PLUS CHRONOPHAGES")
print("="*60)
print(df_taches.to_string(index=False))
df_taches


⏱️ PROCESSUS LES PLUS CHRONOPHAGES
                Processus  Nb Cliniques  Pourcentage
Gestion dossiers patients            22    78.571429
     Prise de rendez-vous            18    64.285714
 Facturation et paiements            18    64.285714
 Gestion stocks/commandes            12    42.857143


,Processus,Nb Cliniques,Pourcentage
1,Gestion dossiers patients,22,78.571429
0,Prise de rendez-vous,18,64.285714
2,Facturation et paiements,18,64.285714
3,Gestion stocks/commandes,12,42.857143


In [4]:
# Visualisation
fig = px.bar(df_taches, x='Pourcentage', y='Processus', orientation='h',
             title='⏱️ Processus les Plus Chronophages',
             labels={'Pourcentage': 'Pourcentage de cliniques (%)', 'Processus': ''},
             color='Pourcentage', color_continuous_scale='Reds',
             text='Pourcentage')
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(height=450, showlegend=False, xaxis_range=[0, 100])
fig.write_html('reports/processus_chronophages.html')
fig.show()
print("✅ Graphique: reports/processus_chronophages.html")

✅ Graphique: reports/processus_chronophages.html


## 🤖 4. Demande d'Automatisation

In [5]:
# Colonnes automatisation
auto_columns = {
    'want_automation_appointments': 'Prise de rendez-vous',
    'want_automation_patient_records': 'Gestion dossiers patients',
    'want_automation_billing': 'Facturation',
    'want_automation_data_analysis': 'Analyse de données',
    'want_automation_inventory': 'Gestion des stocks'
}

auto_data = []
for col, label in auto_columns.items():
    if col in df.columns:
        count = df[col].apply(convert_to_numeric).sum()
        pct = (count / len(df)) * 100
        auto_data.append({
            'Processus': label,
            'Demande': int(count),
            'Pourcentage': pct
        })

df_auto = pd.DataFrame(auto_data).sort_values('Pourcentage', ascending=False)

# Ajouter priorité
def get_priority(pct):
    if pct >= 85: return '🔴 TRÈS HAUTE'
    elif pct >= 70: return '🟡 HAUTE'
    elif pct >= 50: return '🟢 MOYENNE'
    else: return '⚪ FAIBLE'

df_auto['Priorité'] = df_auto['Pourcentage'].apply(get_priority)

print("\n🤖 DEMANDE D'AUTOMATISATION PAR PROCESSUS")
print("="*70)
print(df_auto.to_string(index=False))
df_auto


🤖 DEMANDE D'AUTOMATISATION PAR PROCESSUS
                Processus  Demande  Pourcentage  Priorité
Gestion dossiers patients       23    82.142857   🟡 HAUTE
     Prise de rendez-vous       22    78.571429   🟡 HAUTE
       Analyse de données       15    53.571429 🟢 MOYENNE
       Gestion des stocks       13    46.428571  ⚪ FAIBLE
              Facturation       11    39.285714  ⚪ FAIBLE


,Processus,Demande,Pourcentage,Priorité
1,Gestion dossiers patients,23,82.142857,🟡 HAUTE
0,Prise de rendez-vous,22,78.571429,🟡 HAUTE
3,Analyse de données,15,53.571429,🟢 MOYENNE
4,Gestion des stocks,13,46.428571,⚪ FAIBLE
2,Facturation,11,39.285714,⚪ FAIBLE


In [6]:
# Visualisation demande
fig = px.bar(df_auto, x='Pourcentage', y='Processus', orientation='h',
             title='🤖 Demande d\'Automatisation par Processus',
             labels={'Pourcentage': 'Pourcentage de cliniques (%)', 'Processus': ''},
             color='Pourcentage', color_continuous_scale='Greens',
             text='Pourcentage')
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(height=500, showlegend=False, xaxis_range=[0, 100])
fig.write_html('reports/demande_automatisation.html')
fig.show()
print("✅ Graphique: reports/demande_automatisation.html")

✅ Graphique: reports/demande_automatisation.html


## 📊 5. Matrice Effort vs Impact

In [7]:
# Créer matrice effort/impact
processes_matrix = [
    {'Processus': 'Prise de RDV', 'Impact': 95, 'Effort': 30, 'ROI_mois': 4, 'Gain_h_semaine': 8},
    {'Processus': 'Facturation', 'Impact': 85, 'Effort': 25, 'ROI_mois': 3, 'Gain_h_semaine': 4},
    {'Processus': 'Dossiers Patients', 'Impact': 90, 'Effort': 70, 'ROI_mois': 9, 'Gain_h_semaine': 12},
    {'Processus': 'Analyse Données', 'Impact': 80, 'Effort': 65, 'ROI_mois': 8, 'Gain_h_semaine': 5},
    {'Processus': 'Gestion Stocks', 'Impact': 70, 'Effort': 60, 'ROI_mois': 10, 'Gain_h_semaine': 3},
    {'Processus': 'Rappels SMS', 'Impact': 75, 'Effort': 15, 'ROI_mois': 2, 'Gain_h_semaine': 2}
]

df_matrix = pd.DataFrame(processes_matrix)

# Catégoriser
def categorize_matrix(row):
    if row['Effort'] < 40 and row['Impact'] >= 80:
        return '🟢 Quick Win'
    elif row['Effort'] >= 60 and row['Impact'] >= 80:
        return '🟡 Stratégique'
    elif row['Effort'] < 40 and row['Impact'] < 80:
        return '🔵 Gain Marginal'
    else:
        return '🔴 À Reporter'

df_matrix['Catégorie'] = df_matrix.apply(categorize_matrix, axis=1)

print("\n📊 MATRICE EFFORT VS IMPACT")
print("="*80)
print(df_matrix.to_string(index=False))


📊 MATRICE EFFORT VS IMPACT
        Processus  Impact  Effort  ROI_mois  Gain_h_semaine       Catégorie
     Prise de RDV      95      30         4               8     🟢 Quick Win
      Facturation      85      25         3               4     🟢 Quick Win
Dossiers Patients      90      70         9              12   🟡 Stratégique
  Analyse Données      80      65         8               5   🟡 Stratégique
   Gestion Stocks      70      60        10               3    🔴 À Reporter
      Rappels SMS      75      15         2               2 🔵 Gain Marginal


In [8]:
# Visualisation matrice
color_map = {
    '🟢 Quick Win': '#2ecc71',
    '🟡 Stratégique': '#f39c12',
    '🔵 Gain Marginal': '#3498db',
    '🔴 À Reporter': '#e74c3c'
}

fig = px.scatter(df_matrix, x='Effort', y='Impact', 
                 size='Gain_h_semaine', color='Catégorie',
                 text='Processus', size_max=60,
                 title='📊 Matrice Effort vs Impact',
                 labels={'Effort': 'Effort d\'implémentation', 'Impact': 'Impact sur l\'efficacité'},
                 color_discrete_map=color_map)

fig.update_traces(textposition='top center')
fig.add_shape(type="line", x0=50, y0=0, x1=50, y1=100, line=dict(color="gray", dash="dash"))
fig.add_shape(type="line", x0=0, y0=75, x1=100, y1=75, line=dict(color="gray", dash="dash"))
fig.update_layout(height=600, xaxis_range=[0, 100], yaxis_range=[0, 100])
fig.write_html('reports/matrice_effort_impact.html')
fig.show()
print("✅ Graphique: reports/matrice_effort_impact.html")

✅ Graphique: reports/matrice_effort_impact.html


## 🎯 6. Priorisation et Roadmap

In [9]:
# Trier par priorité (Quick Wins d'abord, puis Stratégiques)
df_priority = df_matrix.sort_values(['Catégorie', 'Impact'], ascending=[True, False])

print("\n🎯 ROADMAP PRIORISÉE")
print("="*80)

for cat in ['�� Quick Win', '🟡 Stratégique', '🔵 Gain Marginal', '🔴 À Reporter']:
    processes = df_priority[df_priority['Catégorie'] == cat]
    if len(processes) > 0:
        print(f"\n{cat}:")
        for _, p in processes.iterrows():
            print(f"  • {p['Processus']} - Impact: {p['Impact']}% | Effort: {p['Effort']}% | ROI: {p['ROI_mois']}mois | Gain: {p['Gain_h_semaine']}h/sem")


🎯 ROADMAP PRIORISÉE

🟡 Stratégique:
  • Dossiers Patients - Impact: 90% | Effort: 70% | ROI: 9mois | Gain: 12h/sem
  • Analyse Données - Impact: 80% | Effort: 65% | ROI: 8mois | Gain: 5h/sem

🔵 Gain Marginal:
  • Rappels SMS - Impact: 75% | Effort: 15% | ROI: 2mois | Gain: 2h/sem

🔴 À Reporter:
  • Gestion Stocks - Impact: 70% | Effort: 60% | ROI: 10mois | Gain: 3h/sem


## 💰 7. Calcul des Gains Potentiels

In [10]:
# Calculer gains par processus
print("\n💰 GAINS ESTIMÉS PAR PROCESSUS")
print("="*80)

for _, row in df_matrix.iterrows():
    processus = row['Processus']
    gain_h = row['Gain_h_semaine']
    nb_cliniques = int(df_auto[df_auto['Processus'].str.contains(processus.split()[0], case=False, na=False)]['Demande'].values[0] if len(df_auto[df_auto['Processus'].str.contains(processus.split()[0], case=False, na=False)]) > 0 else 20)
    
    gain_mensuel_h = gain_h * 4 * nb_cliniques
    cout_horaire = 2500  # FCFA
    economie_mensuelle = gain_mensuel_h * cout_horaire
    economie_annuelle = economie_mensuelle * 12
    
    print(f"\n{processus}:")
    print(f"  • Gain/clinique: {gain_h}h/semaine ({gain_h*4}h/mois)")
    print(f"  • Cliniques cibles: {nb_cliniques}")
    print(f"  • Gain total: {gain_mensuel_h}h/mois")
    print(f"  • Économie mensuelle: {economie_mensuelle:,.0f} FCFA")
    print(f"  • Économie annuelle: {economie_annuelle:,.0f} FCFA")

# Total
total_gain_h = df_matrix['Gain_h_semaine'].sum() * 4 * 20  # Moyenne 20 cliniques
total_economie = total_gain_h * cout_horaire * 12
print(f"\n{'='*80}")
print(f"POTENTIEL TOTAL: {total_gain_h}h/mois | {total_economie:,.0f} FCFA/an")


💰 GAINS ESTIMÉS PAR PROCESSUS

Prise de RDV:
  • Gain/clinique: 8h/semaine (32h/mois)
  • Cliniques cibles: 22
  • Gain total: 704h/mois
  • Économie mensuelle: 1,760,000 FCFA
  • Économie annuelle: 21,120,000 FCFA

Facturation:
  • Gain/clinique: 4h/semaine (16h/mois)
  • Cliniques cibles: 11
  • Gain total: 176h/mois
  • Économie mensuelle: 440,000 FCFA
  • Économie annuelle: 5,280,000 FCFA

Dossiers Patients:
  • Gain/clinique: 12h/semaine (48h/mois)
  • Cliniques cibles: 23
  • Gain total: 1104h/mois
  • Économie mensuelle: 2,760,000 FCFA
  • Économie annuelle: 33,120,000 FCFA

Analyse Données:
  • Gain/clinique: 5h/semaine (20h/mois)
  • Cliniques cibles: 15
  • Gain total: 300h/mois
  • Économie mensuelle: 750,000 FCFA
  • Économie annuelle: 9,000,000 FCFA

Gestion Stocks:
  • Gain/clinique: 3h/semaine (12h/mois)
  • Cliniques cibles: 23
  • Gain total: 276h/mois
  • Économie mensuelle: 690,000 FCFA
  • Économie annuelle: 8,280,000 FCFA

Rappels SMS:
  • Gain/clinique: 2h/semain

## 📊 8. Export des Résultats

In [11]:
# Exporter
df_taches.to_csv('data/processus_chronophages.csv', index=False)
df_auto.to_csv('data/demande_automatisation.csv', index=False)
df_matrix.to_csv('data/matrice_effort_impact.csv', index=False)

print("✅ Données exportées:")
print("  - processus_chronophages.csv")
print("  - demande_automatisation.csv")
print("  - matrice_effort_impact.csv")

✅ Données exportées:
  - processus_chronophages.csv
  - demande_automatisation.csv
  - matrice_effort_impact.csv


## 📝 9. Conclusions et Recommandations

### 🎯 Conclusions Clés

1. **Quick Wins Identifiés**
   - Prise de RDV: 95% d'impact, 30% d'effort → À déployer immédiatement
   - Facturation: 85% d'impact, 25% d'effort → Déploiement rapide
   - ROI de 2-4 mois pour ces solutions

2. **Projets Stratégiques**
   - Dossiers Patients: 90% d'impact, 70% d'effort → Phase 2
   - Analyse de Données: 80% d'impact, 65% d'effort → Phase 2
   - ROI de 8-10 mois mais gains majeurs

3. **Gains Potentiels**
   - 20-35 heures/semaine libérées par clinique
   - Équivalent à 1 ETP récupéré
   - Économies de 3-5 millions FCFA/an par clinique

### 🚀 Recommandations

#### Phase 1 (Mois 1-3): Quick Wins
1. ✅ Déployer module RDV en ligne (15-20 cliniques)
2. ✅ Déployer module Facturation (15-20 cliniques)
3. ✅ Mesurer les gains et créer des cas d'usage

#### Phase 2 (Mois 4-9): Stratégique
1. ✅ Développer et déployer DPE (10-15 cliniques)
2. ✅ Intégrer l'analyse de données (8-12 cliniques)
3. ✅ Optimiser les workflows

#### Phase 3 (Mois 10-18): Optimisation
1. ✅ Gestion intelligente des stocks
2. ✅ Intégration complète des systèmes
3. ✅ IA prédictive et automatisation avancée

---

**L'automatisation n'est plus une option, c'est une nécessité ! 🚀**